In [3]:
list_a=['미세 플라스틱 없는 치약', '미세플라스틱치약', '플라스틱 프리','도전! 플라스틱제로', '플라스틱 줄이기', '재생플라스틱', '친환경플라스틱', '생분해플라스틱', '생분해성 플라스틱', '바이오 플라스틱', '실리콘 빨대', '지구의 날', '대나무 칫솔', '양말목 공예', '업사이클링', '고체치약', '천연 수세미', '샴푸바', '제로 웨이스트', '미니멀 라이프', '플라스틱제로', '제로웨이스트', '양말목공예', '업사이클링제품', '대나무칫솔', '업사이클링', '친환경칫솔', '친환경브랜드', '친환경플라스틱', '친환경선물', '밀랍랩', '친환경쇼핑몰', '실리콘빨대', '고체치약', '생분해플라스틱', '천연수세미']

In [5]:
import pandas as pd
import numpy as np
from pprint import pprint
import urllib.request
import datetime
import json
import glob
import sys
import os
import datetime as dt

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

pd.options.display.float_format = '{:.2f}'.format

In [6]:
class NaverDataLabOpenAPI():
    """
    네이버 데이터랩 오픈 API 컨트롤러 클래스
    """

    def __init__(self, client_id, client_secret):
        """
        인증키 설정 및 검색어 그룹 초기화
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.keywordGroups = []
        self.url = "https://openapi.naver.com/v1/datalab/search"

    def add_keyword_groups(self, group_dict):
        """
        검색어 그룹 추가
        """

        keyword_gorup = {
            'groupName': group_dict['groupName'],
            'keywords': group_dict['keywords']
        }
        
        self.keywordGroups.append(keyword_gorup)
        print(f">>> Num of keywordGroups: {len(self.keywordGroups)}")
        
    def get_data(self, startDate, endDate, timeUnit, device, ages, gender):
        """
        요청 결과 반환
        timeUnit - 'date', 'week', 'month'
        device - None, 'pc', 'mo'
        ages = [], ['1' ~ '11']
        gender = None, 'm', 'f'
        """

        # Request body
        body = json.dumps({
            "startDate": startDate,
            "endDate": endDate,
            "timeUnit": timeUnit,
            "keywordGroups": self.keywordGroups,
            "device": device,
            "ages": ages,
            "gender": gender
        }, ensure_ascii=False)
        
        # Results
        request = urllib.request.Request(self.url)
        request.add_header("X-Naver-Client-Id",self.client_id)
        request.add_header("X-Naver-Client-Secret",self.client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            # Json Result
            result = json.loads(response.read())
            
            period = []
            ratio = []
            title = []
            pprint(result)
            
            if result['results'][0]['data']:
                for val in result['results']:
                    period.append(val['data'][-1]['period'])
                    ratio.append(val['data'][-1]['ratio'])
                    title.append(val['title'])
            
                
            
            df = pd.DataFrame({"title" : title, 'period' : period, "ratio" : ratio})

                
            
        else:
            print("Error Code:" + rescode)
            
        return df

In [7]:
keyword_group_set = {
    'keyword_group_1': {'groupName': "애플", 'keywords': ["애플"]},
}


In [8]:
##날짜지정
today = datetime.date.today()
monday=today - datetime.timedelta(days=today.weekday())
monday=monday.strftime('%Y-%m-%d')
now=dt.datetime.now()
end=monday
print(end)
th=dt.timedelta(days=30)
start=now-th
start=start.strftime('%Y-%m-%d')

2022-08-22


In [9]:
cnt=0
while(True):
    if len(list_a)>0 :
        for idx, (key, val) in enumerate(keyword_group_set.items()):
            val['groupName'] = list_a[idx]
            val['keywords'] = [list_a[idx]]
        del list_a[:1]
        cnt+=1
    else:
        break
#     pprint(keyword_group_set)
    
        # API 인증 정보 설정
    client_id = "xTsOEpqU869lYEaqyaU6"
    client_secret = "kkp_ZP2j9T"

    # 요청 파라미터 설정
    startDate = start
    endDate = end
    timeUnit = 'week'
    device = ''
    ages = []
    gender = ''
    
    # 데이터 프레임 정의
    naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)

    naver.add_keyword_groups(keyword_group_set['keyword_group_1'])

    df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)
    
    if(cnt>1):
        df1=pd.concat([df1,df])
    else:
        df1=df
    

>>> Num of keywordGroups: 1
{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-08-08', 'ratio': 100}],
              'keywords': ['미세 플라스틱 없는 치약'],
              'title': '미세 플라스틱 없는 치약'}],
 'startDate': '2022-07-25',
 'timeUnit': 'week'}
>>> Num of keywordGroups: 1
{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-08-08', 'ratio': 100},
                       {'period': '2022-08-22', 'ratio': 37.5}],
              'keywords': ['미세플라스틱치약'],
              'title': '미세플라스틱치약'}],
 'startDate': '2022-07-25',
 'timeUnit': 'week'}
>>> Num of keywordGroups: 1
{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-07-25', 'ratio': 80.70175},
                       {'period': '2022-08-01', 'ratio': 43.85964},
                       {'period': '2022-08-08', 'ratio': 100},
                       {'period': '2022-08-15', 'ratio': 98.24561},
                       {'period': '2022-08-22', 'ratio': 31.57894}],
              'keywords': ['플라스틱 프리'],
         

{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-07-25', 'ratio': 85.46588},
                       {'period': '2022-08-01', 'ratio': 92.07592},
                       {'period': '2022-08-08', 'ratio': 93.42978},
                       {'period': '2022-08-15', 'ratio': 100},
                       {'period': '2022-08-22', 'ratio': 57.85771}],
              'keywords': ['미니멀 라이프'],
              'title': '미니멀 라이프'}],
 'startDate': '2022-07-25',
 'timeUnit': 'week'}
>>> Num of keywordGroups: 1
{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-07-25', 'ratio': 71.08433},
                       {'period': '2022-08-01', 'ratio': 50.6024},
                       {'period': '2022-08-08', 'ratio': 100},
                       {'period': '2022-08-15', 'ratio': 77.10843},
                       {'period': '2022-08-22', 'ratio': 49.39759}],
              'keywords': ['플라스틱제로'],
              'title': '플라스틱제로'}],
 'startDate': '2022-07-25',
 'timeUnit': 'week'}
>>>

In [10]:
df1

,title,period,ratio
0,미세 플라스틱 없는 치약,2022-08-08,100.00
0,미세플라스틱치약,2022-08-22,37.50
0,플라스틱 프리,2022-08-22,31.58
0,플라스틱 줄이기,2022-08-22,100.00
0,재생플라스틱,2022-08-22,49.66
0,친환경플라스틱,2022-08-22,57.47
0,생분해플라스틱,2022-08-22,54.03
0,생분해성 플라스틱,2022-08-22,89.61
0,바이오 플라스틱,2022-08-22,63.66
0,실리콘 빨대,2022-08-22,72.77


In [58]:
df1=df1[df1['period']==end]

In [59]:
df1

,title,period,ratio
0,제로 웨이스트비누,2022-08-22,89.19
0,제로 웨이스트 챌린지,2022-08-22,61.45
0,제로웨이스트 수세미,2022-08-22,61.17
0,부산 제로웨이스트,2022-08-22,45.45
0,대구 제로웨이스트,2022-08-22,70.97
0,제로웨이스트 실천,2022-08-22,61.41
0,캘러웨이 로스트볼,2022-08-22,51.08
0,제로웨이스트 숙소,2022-08-22,0.42
0,제로웨이스트 뜻,2022-08-22,72.92
0,제로웨이스트 키트,2022-08-22,58.63


In [15]:
df2=df1.sort_values('ratio',ascending=False)
df2=df2.drop([df2.columns[2]],axis=1)

In [16]:
df2.reset_index(drop=True,inplace=True)
df2

,title,period
0,미세 플라스틱 없는 치약,2022-08-08
1,플라스틱 줄이기,2022-08-22
2,친환경쇼핑몰,2022-08-15
3,친환경브랜드,2022-08-22
4,지구의 날,2022-08-22
5,친환경선물,2022-08-22
6,생분해성 플라스틱,2022-08-22
7,업사이클링,2022-08-22
8,업사이클링,2022-08-22
9,업사이클링제품,2022-08-22


In [66]:
data=df2.loc[:9]
data

,title,period,ratio
0,친환경브랜드,2022-08-22,100.00
1,지구의날행사,2022-08-22,100.00
2,친환경제품,2022-08-22,100.00
3,친환경신발,2022-08-22,100.00
4,친환경의류,2022-08-22,93.40
5,제로 웨이스트비누,2022-08-22,89.19
6,업사이클링,2022-08-22,81.35
7,후드티,2022-08-22,81.08
8,제주도수학여행,2022-08-22,80.84
9,리사이클링,2022-08-22,77.17


In [3]:
dt.datetime.now()

datetime.datetime(2022, 8, 26, 10, 46, 14, 983210)

In [18]:
now=dt.datetime.now()
now

datetime.datetime(2022, 8, 26, 11, 12, 25, 103654)

In [20]:
startDate=now.strftime('%Y-%m-%d')
startDate

'2022-08-26'

In [21]:
th=dt.timedelta(days=30)
endDate=day-th


In [22]:
th=dt.timedelta(days=30)
endDate=day-th
endDate=endDate.strftime('%Y-%m-%d')
endDate

'2022-07-27'

In [51]:
today = datetime.date.today()
monday=today - datetime.timedelta(days=today.weekday())
monday

datetime.date(2022, 8, 22)

In [ ]:
startDate = "2022-08-01"
endDate = "2022-08-25"

NameError: name 'driver' is not defined